In [ ]:
from atlassian import Confluence
import requests
import random
from bs4 import BeautifulSoup
import base64

In [ ]:
user = 'myemail@gmail.com'  # Replace with your Confluence username
apikey = 'your_API_Token'  # Replace with your actual API key
server = 'url to your space' #Replace with your URL to your Confluence space

In [ ]:
confluence = Confluence(
    url=server,
    username=user,
    password=apikey)

In [ ]:
def get_pages_from_space(space_key):
    limit = 100
    pages = []
    start = 0

    while True:
        response = confluence.get_all_pages_from_space(
            space=space_key, start=start, limit=limit, expand='body.storage')
        if not response:
            break
        pages.extend(response)
        start += limit

    return pages

In [ ]:
def page_contains_images(page):
    content = page['body']['storage']['value']
    soup = BeautifulSoup(content, 'html.parser')
    return bool(soup.find_all('ac:image'))

In [ ]:
def page_contains_tables(page):
    content = page['body']['storage']['value']
    soup = BeautifulSoup(content, 'html.parser')
    return bool(soup.find_all('table'))

In [ ]:
space_key = 'space key' #Replace with your space key
try:
    pages = get_pages_from_space(space_key)
except ApiPermissionError as e:
    print("Permission error:", e)
    print("Please check the user's permissions and the space key.")
    raise

In [ ]:
pages_with_images = [page for page in pages if page_contains_images(page)]
pages_with_tables = [page for page in pages if page_contains_tables(page)]
pages_without_images_or_tables = [page for page in pages if not page_contains_images(page) and not page_contains_tables(page)]

In [ ]:
print(f"Total pages: {len(pages)}")
print(f"Pages with images: {len(pages_with_images)}")
print(f"Pages with tables: {len(pages_with_tables)}")
print(f"Pages without images or tables: {len(pages_without_images_or_tables)}")

In [ ]:
if len(pages_with_images) < 2 or len(pages_with_tables) < 2 or len(pages_without_images_or_tables) < 2:
    raise ValueError("Not enough pages to meet the selection criteria.")

In [ ]:
selected_with_images = random.sample(pages_with_images, 1)
selected_with_tables = random.sample(pages_with_tables, 1)
selected_without_images_or_tables = random.sample(pages_without_images_or_tables, 2)

In [ ]:
selected_pages = selected_with_images + selected_with_tables + selected_without_images_or_tables

In [ ]:
random.shuffle(selected_pages)

In [ ]:
print("Selected Pages:")
for page in selected_pages:
    print(page['title'])